In [1]:
%pip install -q --upgrade langchain langchain-experimental langchain-openai python-dotenv pyvis

Note: you may need to restart the kernel to use updated packages.


# Set up Phoenix

In [2]:
from phoenix.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
import os
from opentelemetry.trace import Status, StatusCode
from openinference.semconv.trace import SpanAttributes

project_name = "Knowledge_base_QA"

# Add Phoenix API Key for tracing
phoenix_key = ''
with open('phoenix_key.txt', 'r') as file:
    phoenix_key = file.read()
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={phoenix_key}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={phoenix_key}";
os.environ['PHOENIX_PROJECT_NAME'] = project_name

# configure the Phoenix tracer
tracer_provider = register(
  project_name=project_name, # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

OpenAIInstrumentor().instrument(tracer_provider = tracer_provider)
tracer = tracer_provider.get_tracer(__name__)

Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Knowledge_base_QA
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



# Basic imports and setups

In [3]:
from dotenv import load_dotenv
import os
import datetime
import json
from time import sleep
import numpy as np
import pandas as pd
from pydantic import BaseModel, Field
from pybars import Compiler
import yaml

from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

from mlx_lm import load, generate

# import own utility functions
from llm_graph import *
from pdf_preprocessing import *


# settings
model_type = "openrouter"
ticker = 'BHP'
ticker_profile = "BHP Group Limited operates as a resources company in Australia, Europe, China, Japan, India, South Korea, the rest of Asia, North America, South America, and internationally. The company operates through Copper, Iron Ore, and Coal segments. It engages in the mining of copper, uranium, gold, zinc, lead, molybdenum, silver, iron ore, cobalt, and metallurgical and energy coal. The company is also involved in the mining, smelting, and refining of nickel, as well as potash development activities. In addition, it provides towing, freight, marketing and trading, marketing support, finance, administrative, and other services. The company was founded in 1851 and is headquartered in Melbourne, Australia."
version = '1.6.0'

# Cut the input text to paragraph, if False it will cut to PDF pages
cut_in_paragraph = False
template_file_path = './prompts'

# LLM Model

In [4]:
# we use MLX_LLM in the background

if model_type == "mlx":
    # local models
    model_name = 'mlx-community/Meta-Llama-3.1-8B-Instruct-4bit'
    # 8 bit cab fit in M4 memmory but seems the 4bit enough for our task
    # so do not justfly the double memory usage
    # model_name = 'mlx-community/Meta-Llama-3.1-8B-Instruct-8bit')
    # model_name = 'mlx-community/Qwen3-8B-6bit'
    # model_name = 'mlx-community/gemma-3-12b-it-4bit-DWQ')

    api_key = "nem_kell"

    base_url = "http://localhost:8000/v1"
elif model_type == "openrouter":
    # openrouter models
    model_name = "qwen/qwen3-30b-a3b:free"

    with open('openrouter_key.txt', 'r') as file:
        api_key = file.read()

    base_url = "https://openrouter.ai/api/v1"


In [5]:
llm = ChatOpenAI(
    temperature=0, 
    model_name=model_name, 
    openai_api_base=base_url,
    api_key=api_key
)


# Read data

Read PDF financial reports and process them to Langchain Documents

In [6]:
paragraphs = get_paragraphs(ticker)

Processing: knowledge/bhp_20240701_20241231_qa_1.pdf file


In [7]:
# load qa_prompt
# open file
system_propmt_file = os.path.join(template_file_path, 'qa_classification.yaml')
# test file exist
if not os.path.isfile(system_propmt_file):
    raise ValueError(f"{system_propmt_file} not a valid file")
with open(system_propmt_file) as file:
    source = yaml.safe_load(file)
# Compile the template
compiler = Compiler()
qa_prompt_template = compiler.compile(source['prompt_template'])
qa_propmt = qa_prompt_template({})

In [8]:
# we will use this for quality testing in Phoenix
data_sample = []

# itterate over the paragrpahs and decide what category they are
stepsize = 3
with tracer.start_as_current_span(f"qa_category_{datetime.datetime.now().timestamp()}") as parent_span: 
    for pidx in range(stepsize, len(paragraphs)+1):
        print(f"Progress: {np.round(100*pidx/len(paragraphs),2)}%", end="\r")
        this_section = paragraphs[pidx-stepsize:pidx]
        text_json = {
            "COMPANY": ticker,
            "COMPANY_DESCRIPTION": ticker_profile
        }
        for p in range(len(this_section)):
            if "qa_type" not in this_section[p].metadata:
                this_section[p].metadata['qa_type'] = []
            text_json[f"text_{p}"] = this_section[p].page_content
        data_sample.append(text_json)
    
        messages = [
            ("system", qa_propmt),
            ("user", json.dumps(text_json, indent=4))
        ]

        # Phoenix span
        with tracer.start_as_current_span("child") as child_span:
            child_span.set_attribute(SpanAttributes.INPUT_VALUE, json.dumps(text_json, indent=4))
            child_span.set_attribute(SpanAttributes.LLM_MODEL_NAME, model_name)

            try:
                response = llm.with_structured_output(QAType).invoke(messages)
            except Exception as e:
                child_span.set_attribute(SpanAttributes.OUTPUT_VALUE, str(e))
                child_span.set_status(Status(StatusCode.ERROR))
                continue

            response = dict(response)
            child_span.set_attribute(SpanAttributes.OUTPUT_VALUE, json.dumps(response, indent=4))

            for p in range(len(this_section)):
                this_section[p].metadata['qa_type'].append(response[f"text_{p+1}_class"])

            child_span.set_status(Status(StatusCode.OK))
    # we use a free model so there is limited 
    sleep(1)

Progress: 100.0%

# Graph

In [9]:
filtered_paragraphs = []

for pidx in range(len(paragraphs)):
    # clean QA category
    qa_category, counts = np.unique(paragraphs[pidx].metadata['qa_type'], return_counts=True)

    qa_category = { 
        str(category): int(count) 
        for category, count in zip(qa_category, counts)
        if str(category) != "NONE_OF_ABOVE"
    }

    if len(qa_category.values()) == 1:
        this_paragraphs = paragraphs[pidx].model_copy()
        this_paragraphs.metadata['qa_type'] = list(qa_category.keys())[0]
        filtered_paragraphs.append(this_paragraphs)

print(len(filtered_paragraphs))

14


In [10]:
from llm_graph import *

# Define the allowed node schemas
node_schemas = [
    NodeSchema(
        "Person",
        ["name", "birth_year", "profession"],
        "Represents an individual human being. Examples include employees, entrepreneurs, or public figures."
    ),
    NodeSchema(
        "Company",
        ["name"],
        "Represents a business entity or organization. This includes attributes such as the company's name and the industry it operates in (e.g., finance, technology, manufacturing). Examples include multinational corporations, startups, or small businesses."
    ),
    NodeSchema(
        "Product",
        ["name"],
        "Represents a tangible or intangible item that is created, manufactured, or offered by a company. This includes goods such as electronics, clothing, or software. A Product's name should always be a noun."
    ),
    NodeSchema(
        "Activity",
        ["name"],
        "Represents a specific business activity or operation performed by a company. This could include manufacturing processes, service offerings, or other business-related activities such as acquisitions, partnerships, or marketing campaigns. Examples include 'Car Manufacturing', 'Software Developing', or 'Market Expansion'. An Activity's name should always be a verb."
    ),
    NodeSchema(
        "Location",
        ["name"],
        "Represents a geographical area, such as a city, country, or region. This includes physical locations where companies operate, products are manufactured, or activities take place. Examples include 'New York City', 'Germany', or 'Asia-Pacific Region'."
    ),
]
    
# Define the allowed relationship schemas
relationship_schemas = [
    RelationshipSchema("Person", "WORK_FOR", "Company", ["start_year", "end_year", "year"]),
    RelationshipSchema("Company", "OPERATE_IN", "Location"),
    RelationshipSchema("Company", "MADE_BY", "Product", ["start_year", "end_year", "year"]),
    RelationshipSchema("Company", "DOING", "Activity", ["start_year", "end_year", "year"]),
]

additional_instructions = f"""
# Background information about the text

The text is from the {ticker} company investor call transcript. {ticker_profile}. 
In the transcript investors asking question from the {ticker} representatives who answers them.

"""

# Generate the Graph class
Graph = generate_graph_class(node_schemas.copy(), relationship_schemas.copy())

# Load prompt template
node_definitions = format_node_schemas(node_schemas)
node_properties_definitions = format_node_properties_schemas(node_schemas)
relationship_definitions = format_relationship_schemas(relationship_schemas)
relationship_properties_definitions = format_relationship_properties_schemas(relationship_schemas)
compiler = Compiler()
# open file
system_propmt_file = os.path.join(template_file_path, 'knowledge_graph.yaml')
# test file exist
if not os.path.isfile(system_propmt_file):
    raise ValueError(f"{system_propmt_file} not a valid file")
with open(system_propmt_file) as file:
    source = yaml.safe_load(file)
# Compile the template
knowledge_graph_template = compiler.compile(source['prompt_template'])

graph_documents_nodes_defined = []

with tracer.start_as_current_span(f"graphs_{datetime.datetime.now().timestamp()}") as parent_span: 
    for pidx in range(len(filtered_paragraphs)):
        print(f"Progress: {np.round(100*(pidx+1)/len(filtered_paragraphs),2)}%", end="\r")
        paragraph = filtered_paragraphs[pidx]
        formatted_prompt = knowledge_graph_template({
            "node_definitions": node_definitions,
            "relationship_definitions": relationship_definitions,
            "node_properties_definitions": node_properties_definitions,
            "relationship_properties_definitions": relationship_properties_definitions,
            "additional_instructions": additional_instructions,
            "input_text": paragraph.page_content
        })

        messages = [
            ("user", formatted_prompt)
        ]

        with tracer.start_as_current_span("child") as child_span:
            child_span.set_attribute(SpanAttributes.INPUT_VALUE, formatted_prompt)
            child_span.set_attribute(SpanAttributes.LLM_MODEL_NAME, model_name)
            try:
                result = llm.with_structured_output(Graph).invoke(messages)
            except Exception as e:
                child_span.set_attribute(SpanAttributes.OUTPUT_VALUE, str(e))
                child_span.set_status(Status(StatusCode.ERROR))
                continue

            enriched_graph = add_paragraph_node(
                result,
                paragraph.page_content, 
                title=paragraph.metadata["filename"], 
                filename=paragraph.metadata["filename"]
            )
            graph_documents_nodes_defined.append(enriched_graph)
            child_span.set_attribute(SpanAttributes.OUTPUT_VALUE, json.dumps(result.model_dump(), indent=4))
            child_span.set_status(Status(StatusCode.OK))
    
        sleep(1)


Progress: 100.0%

In [11]:
# merge the list of Graph together
merged_graph_documents_nodes_defined = merge_graphs(graph_documents_nodes_defined)

In [12]:
# clean up the nodes
clean_graph = merge_nodes(merged_graph_documents_nodes_defined)
clean_graph

'europe'
'china'
'japan'
'india'
'south korea'
'rest of asia'
'north america'
'south america'
'international'
'copper'
'iron ore'
'coal'
'uranium'
'gold'
'zinc'
'lead'
'molybdenum'
'silver'
'cobalt'
'metallurgical and energy coal'


Graph(nodes=[Node(id='BHP Group Limited', type='Company', properties={'name': 'BHP Group Limited'}), Node(id='Australia', type='Location', properties={'name': 'Australia'}), Node(id='Europe', type='Location', properties={'name': 'Europe'}), Node(id='China', type='Location', properties={'name': 'China'}), Node(id='Japan', type='Location', properties={'name': 'Japan'}), Node(id='India', type='Location', properties={'name': 'India'}), Node(id='South Korea', type='Location', properties={'name': 'South Korea'}), Node(id='Asia-Pacific Region', type='Location', properties={'name': 'Asia-Pacific Region'}), Node(id='North America', type='Location', properties={'name': 'North America'}), Node(id='South America', type='Location', properties={'name': 'South America'}), Node(id='International', type='Location', properties={'name': 'International'}), Node(id='Copper', type='Product', properties={'name': 'Copper'}), Node(id='Iron Ore', type='Product', properties={'name': 'Iron Ore'}), Node(id='Coal',

# Visualization

In [13]:
visualize_graph(clean_graph)

Graph saved to /Users/Attila_Sajo/Git/blog/Knowledge_base/knowledge_graph.html


<class 'pyvis.network.Network'> |N|=92 |E|=231

# Load to Kuzu

In [14]:
import numpy as np
import pandas as pd
import kuzu
import hashlib

# import polars as pl

db = kuzu.Database(":memory:")
db = kuzu.Database()
conn = kuzu.Connection(db)

def load_to_kuzu(graph: Graph):
    nodes = graph.nodes
    relations = graph.relationships

    # get unique node types
    node_types = set([ node.type for node in nodes ])
    # itterate over node types and create individual DataFrames for load
    for node_type in node_types:
        # filter the nodes
        this_nodes = [ node for node in nodes if node.type == node_type ]
        # detect primery key
        primary_key = detect_primary_key(this_nodes)
        # the SQL to create the node table
        create_node_table_sql = f"CREATE NODE TABLE IF NOT EXISTS {node_type}(\n"
        # get attibutes
        attirbutes = [ list(node.properties.keys()) for node in this_nodes ]
        attirbutes = set([item for sublist in attirbutes for item in sublist])
        df = pd.DataFrame()
        for attirbute in attirbutes:
            this_attribute = []
            for node in this_nodes:
                if attirbute in list(node.properties.keys()):
                    this_attribute.append(node.properties[attirbute])
                else:
                    this_attribute.append(None)
            df[attirbute] = this_attribute
        # get SQL type from Pandas dtype
        attributes_types = pandas_to_sql_types(df)
        # add attribute columns to SQL table
        for k,v in attributes_types.items():
            create_node_table_sql += f"    {k} {v},\n"
        # if id is the primary key
        if primary_key["primary_key"] is None:
            df['id'] = [ hashlib.md5(str(node.id).encode()).hexdigest() for node in this_nodes ]
        else:
            df['id'] = [ 
                hashlib.md5(str(node.properties[primary_key["primary_key"][0]]).encode()).hexdigest() 
                for node in this_nodes 
            ]
        # close the SQL defination for Node table
        create_node_table_sql += "    id STRING PRIMARY KEY\n);"

        # create node table
        conn.execute(create_node_table_sql)
        # load nodes
        conn.execute(f"COPY {node_type} FROM $dataframe", {"dataframe": df})

    # define relations in the DB
    # get unique relation types
    relation_types = set([ relation.type for relation in relations ])
    for relation_type in relation_types:
        # filter the nodes
        this_relations = [ relation for relation in relations if relation.type == relation_type ]
        # get the relationships where the from and to node types are same
        connections_directions = [ {'source': relation.source_type, 'target': relation.target_type } for relation in this_relations ]
        connections_directions = [dict(t) for t in {tuple(d.items()) for d in connections_directions}]
        for connections_direction in connections_directions:
            table_name = f'{relation_type}_{connections_direction["source"].upper()}_{connections_direction["target"].upper()}'
            create_relation_table_sql = f'CREATE REL TABLE IF NOT EXISTS {table_name}(\n'
            create_relation_table_sql += f'    FROM {connections_direction["source"]} TO {connections_direction["target"]}\n'
            create_relation_table_sql += ");"
            conn.execute(create_relation_table_sql)

            # get the nodes for this df
            this_table_relations = [ 
                relation for relation in this_relations
                if relation.source_type == connections_direction["source"]
                and relation.target_type == connections_direction["target"]
            ]
            df = pd.DataFrame({
                "from": [ hashlib.md5(str(relation.source).encode()).hexdigest() for relation in this_table_relations ],
                "to": [ hashlib.md5(str(relation.target).encode()).hexdigest() for relation in this_table_relations ]
            })
            load_relation_table_sql = f"""
                COPY {table_name} FROM $dataframe (ignore_errors=true)
            """
            conn.execute(load_relation_table_sql, {"dataframe": df})
        
    
load_to_kuzu(clean_graph)

In [15]:
rows = conn.execute("MATCH (p:Person)-[:WORK_FOR_PERSON_COMPANY]->(c:Company) RETURN p.id, p.name, c.name, c.id")
print(rows.get_as_df())

                               p.id             p.name             c.name  \
0  f05db2fb41cfacf96466b4d301672366   Liam Fitzpatrick      Deutsche Bank   
1  1d6d7c8e66cfd9f706ab094dc130df30         Mike Henry  BHP Group Limited   
2  e48a55f3e88d2e94dc9dbb50ef3f4b86   Jason Fairclough    Bank of America   
3  7348b4c64e83411dde284c168d5a908b      Amos Fletcher  BHP Group Limited   
4  20efa746983df86c0c36eeac16a5fb9d       Vandita Pant  BHP Group Limited   
5  4f1d6f45e3eb82dbaf454aeb46483ab4  VANDITA PANT, BHP  BHP Group Limited   
6  1bddabaffa161cace10d4e9ad6527d70        Matt Greene  BHP Group Limited   

                               c.id  
0  54ebc4b3f1a2a0984d8e6fe04e5b873d  
1  f458659825cb5b9141b09d1e658f6132  
2  e4b68a3710443456cf882f506dfa2af5  
3  f458659825cb5b9141b09d1e658f6132  
4  f458659825cb5b9141b09d1e658f6132  
5  f458659825cb5b9141b09d1e658f6132  
6  f458659825cb5b9141b09d1e658f6132  


In [16]:
rows = conn.execute("MATCH (c:Company) RETURN c.name, c.id")
print(rows.get_as_df())

              c.name                              c.id
0  BHP Group Limited  f458659825cb5b9141b09d1e658f6132
1      Deutsche Bank  54ebc4b3f1a2a0984d8e6fe04e5b873d
2             Vicuña  4d2705c724cfc91caa9dc64119d8456e
3    Bank of America  e4b68a3710443456cf882f506dfa2af5
4           Barclays  167cd5762065530b3bf05b9d1f4fed66
5          Westshore  9f3c120af482df44adf99fb395e7471c
6      Goldman Sachs  d81a3a023d342920551254d6f821d7b2


In [17]:
[ node for node in merged_graph_documents_nodes_defined.nodes if "name" in node.properties and node.properties["name"] == "BHP Group Limited" ]

[Node(id='BHP Group Limited', type='Company', properties={'name': 'BHP Group Limited'}),
 Node(id='bhp group limited', type='Company', properties={'name': 'BHP Group Limited'}),
 Node(id='bhp', type='Company', properties={'name': 'BHP Group Limited'})]